In [1]:
import pandas as pd 
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('watchGo').getOrCreate()
sqlContext = SQLContext(spark)

In [34]:
def df_hdfs_spark(data, nombre_csv):
    '''
    Crea un datafram de spark a partir de uno de pandas,
    lo escribe en hdfs y lo vuelve a leer y guarda
    '''
    path = 'hdfs://namenode:8020/' + nombre_csv + '.parquet'
    df_spark = sqlContext.createDataFrame(data=data) #pd.dataframe --> spark
    df_spark.write.parquet(path, mode = 'overwrite') #spark --> hdfs  ojo con el parametro mode!!
    return spark.read.parquet(path) #hdfs --> spark

In [12]:
justwatch = pd.read_csv('data/justwatch.csv')
#imdb = pd.read_csv('data/IMDb movies.csv')
cartelera = pd.read_csv('data/cartelera.csv')

In [35]:
jw = df_hdfs_spark(justwatch, 'jw')

In [36]:
jw.show()

+-----------+--------+--------------------+
| Plataforma|    Tipo|              Titulo|
+-----------+--------+--------------------+
|disney-plus|pelicula|el imperio de los...|
|disney-plus|   serie|     elena de avalor|
|disney-plus|   serie| big hero 6 la serie|
|disney-plus|pelicula|el cascanueces y ...|
|disney-plus|pelicula|              flicka|
|disney-plus|pelicula|  recess schools out|
|disney-plus|pelicula|          magic camp|
|disney-plus|pelicula|el regreso de los...|
|disney-plus|   serie|mas muppets que n...|
|disney-plus|   serie|las aventuras de ...|
|disney-plus|pelicula|marvel construyen...|
|disney-plus|pelicula|   black beauty 2020|
|disney-plus|pelicula|    mighty joe young|
|disney-plus|   serie|               raven|
|disney-plus|pelicula|     el abismo negro|
|disney-plus|   serie|gordons great escape|
|disney-plus|   serie|the world accordi...|
|disney-plus|pelicula|     nada que perder|
|disney-plus|   serie|             mech x4|
|disney-plus|pelicula|          

In [37]:
jw.filter(F.col('Titulo') == 'stranger things').show()

+----------+-----+---------------+
|Plataforma| Tipo|         Titulo|
+----------+-----+---------------+
|   netflix|serie|stranger things|
+----------+-----+---------------+



In [38]:
cine = df_hdfs_spark(cartelera, 'cine')

In [39]:
cine.show()

+--------------------+--------------------+
|            Pelicula|               Cines|
+--------------------+--------------------+
|      Mortal Kombat |['Cine Teatro Gus...|
|Una joven promete...|['Cines Embajador...|
|¡Upsss 2! ¿Y ahor...|['Cines Embajador...|
|Una veterinaria e...|['Cines Embajador...|
|Detective Conan: ...|['Cines La Vaguad...|
|Crock of Gold: Be...|['Cines Verdi', '...|
|       Natura Bizia |           ['Golem']|
| Amanece en Calcuta |['Cines La Vaguad...|
|       Palm Springs |                  []|
|Zog y los doctore...|                  []|
|         Otra ronda |['Cine Estudio (C...|
|            Cuñados |['Cines La Vaguad...|
|       I Care a Lot |['Kinépolis Madri...|
| Guerra de mentiras |                  []|
|            La nube |['Renoir Cines Pr...|
|Uno cero uno, el ...|['Cines La Vaguada']|
|               Vivo |['Cines La Vaguad...|
| Ruega por nosotros |['Cines La Vaguad...|
|Los Estados Unido...|['Cines Verdi', '...|
|Digimon Adventure...|['Cines La

In [14]:
spark.stop()